In [1]:
import math
import numpy as np
from matplotlib import pyplot as plt
from numpy import random as rnd
import warnings,datetime,os,calendar,csv,time
import pickle,h5py,json

import tensorflow as tf
import pandas as pd
import seaborn as sns

from keras.models import Model,Sequential
from keras.layers import Dense,LSTM,Conv2D,Dropout,BatchNormalization,Input,Concatenate,Add,Activation,MaxPooling2D,AveragePooling2D
import keras.backend as K

from sklearn import preprocessing as pp

from sklearn.cluster import KMeans,MeanShift
from sklearn.dummy import DummyClassifier,DummyRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier,AdaBoostRegressor,RandomForestClassifier,RandomForestRegressor
from sklearn.linear_model import BayesianRidge,Lasso,LinearRegression,SGDClassifier,SGDRegressor
from sklearn.mixture import BayesianGaussianMixture,GaussianMixture
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor,RadiusNeighborsClassifier,RadiusNeighborsRegressor,NearestNeighbors
from sklearn.manifold import Isomap,TSNE
from sklearn.model_selection import KFold,StratifiedKFold,train_test_split
from sklearn.svm import LinearSVC,LinearSVR
from sklearn.neural_network import BernoulliRBM,MLPClassifier,MLPRegressor
from sklearn.decomposition import FactorAnalysis,KernelPCA,PCA,MiniBatchSparsePCA,FastICA
from sklearn.preprocessing import CategoricalEncoder,KBinsDiscretizer,LabelEncoder,MinMaxScaler,OneHotEncoder,StandardScaler

from scipy import fftpack
from scipy.signal import resample,correlate
from xgboost import XGBClassifier,XGBRegressor

import gym

import pandas_datareader as pdr
import urllib
from bs4 import BeautifulSoup
import networkx as nx
from lmfit import Model

import iexfinance as iex
from iexfinance.stocks import get_historical_data

warnings.filterwarnings('ignore')
sns.set()
plt.xkcd()

C:\Users\Jan\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [80]:
def generate_set(num_samples,BINMAX=8,NOISE_STD=0.1):
    
    MAXLEN = len(bin(BINMAX)[2:])-1

    p_list = [bin(k)[2:].rjust(MAXLEN,'0') for k in range(BINMAX)]
    symbols = np.array([[int(k) for k in l] for l in p_list])
    
    y = rnd.choice(BINMAX,p=np.ones(symbols.shape[0])/symbols.shape[0],size=[num_samples])
    X = np.array([symbols[k] for k in y])
    
    return X,y

In [197]:
class Channel():
    
    def __init__(self,channel_in_size,channel_out_size,MU_R=-1,SIGMA_R=0.1,MU_H=8,SIGMA_H=0.5,SWAP_PROB_SIGMA=0.1):
        
        self.channel_in_size = channel_in_size
        self.channel_out_size = channel_out_size
        
        self.MU_R = MU_R
        self.SIGMA_R = SIGMA_R
        
        self.MU_H = MU_H
        self.SIGMA_H = SIGMA_H
        self.SWAP_PROB_SIGMA = SWAP_PROB_SIGMA
        
        self.H = self.SIGMA_H*rnd.randn(self.channel_out_size,self.channel_in_size) + self.MU_H
        
        p_swap_c = rnd.rand(self.channel_out_size,self.channel_out_size)
        self.p_swap = self.SWAP_PROB_SIGMA*(np.triu(p_swap_c,1)+np.triu(p_swap_c,1).transpose())
        
        return
        
    def transfer(self,channel_in):
        
        x_in = np.array([int(k) for l in bin(channel_in)[2:].rjust(self.channel_in_size,'0') for k in l])
        
        y_unnoisy = np.dot(self.H,x_in.reshape(-1,1))
        y_noisy = y_unnoisy + (self.SIGMA_R*rnd.randn(self.channel_out_size) + self.MU_R).reshape(-1,1)
        
        for k0 in range(self.p_swap.shape[0]):
            for k1 in range(self.p_swap.shape[1]):
                if rnd.rand()<self.p_swap[k0,k1]: 
                    m0 = y_noisy[k0]
                    m1 = y_noisy[k1]
                    y_noisy[k0] = m1
                    y_noisy[k1] = m0
        
        return x_in,y_noisy

In [198]:
mychannel = Channel(3,2)

In [201]:
mychannel.transfer(1)

(array([0, 0, 1]), array([[7.28136505],
        [7.42846644]]))

NameError: name 'self' is not defined

1 -- 3


In [190]:
y_noisy

array([[6.619898  ],
       [7.46546017],
       [7.73989789],
       [7.46546017],
       [6.46397254]])

In [3]:
X_,y_ = generate_set(3000)

ohe = OneHotEncoder()
y_ohe = ohe.fit_transform(y_.reshape(-1,1)).todense()

X0,X1,y0,y1 = train_test_split(X_,y_ohe,test_size=0.1)

In [63]:
in0 = Input(shape=(2,))
x0 = Dense(units=5,activation='relu')(in0) 
x1 = Dense(units=5,activation='relu')(x0) 
out0 = Dense(units=4,activation='softmax')(x1)

model = Model(inputs=in0,outputs=out0)
#model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['mse'])

TypeError: __init__() missing 1 required positional argument: 'func'

In [60]:
Input(shape=(2,))

<tf.Tensor 'input_8:0' shape=(?, 2) dtype=float32>